In [1]:
!pip install pyautogui
!pip install pyperclip
!pip install selenium
!pip install bs4

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.common.action_chains import ActionChains

# 크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager
import time
import pyautogui
import pyperclip


URL = "https://stephenstone.tistory.com/"  # 네이버 카페 주소
USER_ID = "stephenstone"  # 네이버 아이디
USER_PW = "alsh616!@"  # 네이버 비밀번호
secure='blank'

tags = ['ESG', 'DT', 'DX', '일일브리핑']
title = 'test'
content = 'test'
# 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러메세지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
wait = WebDriverWait(driver, 10)
# 웹페이지 해당 주소 이름
driver.implicitly_wait(5)  # 웹페이지가 로딩될 때까지 5초 기다림
driver.maximize_window()  # 화면 최대화
driver.get("https://accounts.kakao.com/login/?continue=https%3A%2F%2Fkauth.kakao.com%2Foauth%2Fauthorize%3Fis_popup%3Dfalse%26ka%3Dsdk%252F2.7.3%2520os%252Fjavascript%2520sdk_type%252Fjavascript%2520lang%252Fko-KR%2520device%252FWin32%2520origin%252Fhttps%25253A%25252F%25252Fwww.tistory.com%26auth_tran_id%3DgQNeDDVOqs5dQlpPKIv3OOUj~Y5GeRz0zmKidzpLmxRBHUBuiSEsf-M.85.X%26response_type%3Dcode%26state%3DaHR0cHM6Ly93d3cudGlzdG9yeS5jb20v%26redirect_uri%3Dhttps%253A%252F%252Fwww.tistory.com%252Fauth%252Fkakao%252Fredirect%26client_id%3D3e6ddd834b023f24221217e370daed18%26through_account%3Dtrue&talk_login=hidden#login")

# 아이디 입력창
id = driver.find_element(By.CSS_SELECTOR, "#mainContent > div > div > form > div.item_form.fst > div")
id.click()
pyperclip.copy(USER_ID)
pyautogui.hotkey("ctrl", "v")
time.sleep(2)

# 비밀번호 입력창
pw = driver.find_element(By.CSS_SELECTOR, "#password--2")
pw.click()
pyperclip.copy(USER_PW)
pyautogui.hotkey("ctrl", "v")
time.sleep(2)
pyperclip.copy(secure)  #비밀번호 보안을 위해 클립보드에 blank 저장
        

# 로그인 버튼
driver.find_element(By.XPATH,'//*[@id="mainContent"]/div/div/form/div[4]/button[1]').click()
time.sleep(2)


driver.get(URL)



# 글쓰기 버튼 클릭 (카페별로 버튼 위치가 다를 수 있음)
##CSS selector : cafe-info-data > div.cafe-write-btn > a
element = driver.find_element(By.CSS_SELECTOR, "#container > main > aside > div.box-profile > div > div.profile-btn-group.member > button.btn-g.btn-primary.btn-write")
element.click()  # 클릭 이벤트 수행 (필요시)
time.sleep(3)

try:
    # 모달창(Alert)으로 전환 시도
    alert = driver.switch_to.alert
    print("모달창이 있습니다. 확인 버튼을 클릭합니다.")
    alert.dismiss()  # 확인 버튼 클릭
except NoAlertPresentException:
    print("모달창이 존재하지 않습니다. 다음 단계로 넘어갑니다.")

# # 새로운 창으로 전환
# WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)  # 새 창 열리기를 대기
# new_window = driver.window_handles[-1]  # 새 창 핸들 (맨 마지막 창)
# driver.switch_to.window(new_window)  # 새 창으로 전환

# 드롭다운 버튼 클릭
dropdown_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#editor-mode-layer-btn-open"))
)
dropdown_button.click()

# 'html' 옵션 선택
dropdown_option = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="editor-mode-html"]'))
)
dropdown_option.click()

# 모달창(브라우저 Alert)으로 전환
alert = driver.switch_to.alert

# 확인 버튼 클릭
alert.accept()

# 드롭다운 버튼 클릭
dropdown_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#category-btn"))
)
dropdown_button.click()

# '일일 브리핑' 옵션 선택
dropdown_option = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="category-item-1131400"]'))
)
dropdown_option.click()

# 제목 입력

title_field = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#post-title-inp"))
)
title_field.clear()  # 기존 내용 삭제
title_field.send_keys(title)

driver.execute_script("""
    var editor = document.querySelector('.CodeMirror').CodeMirror;
    editor.setValue(arguments[0]);
""", content)

# 테그 입력
tag_input = driver.find_element(By.CSS_SELECTOR, "#tagText")
    # TAG 리스트를 순회하며 입력
for tag in tags:
    tag_input.send_keys(tag)  # 태그 입력
    tag_input.send_keys(Keys.ENTER)  # 엔터 키 입력
    time.sleep(1)  # 입력 후 약간의 딜레이 (필요 시 조정)
    
print("모든 태그가 입력되었습니다.")

# 등록 버튼 클릭
submit_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#publish-layer-btn"))
)
submit_button.click()  # 등록 버튼 클릭

submit_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#publish-btn"))
)
submit_button.click()  # 등록 버튼 클릭


print("게시글이 성공적으로 등록되었습니다!")







모달창이 있습니다. 확인 버튼을 클릭합니다.


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=131.0.6778.140)
Stacktrace:
	GetHandleVerifier [0x00A634A3+25059]
	(No symbol) [0x009ECEA4]
	(No symbol) [0x008CBD39]
	(No symbol) [0x0090AC49]
	(No symbol) [0x00909ABD]
	(No symbol) [0x00931EFC]
	(No symbol) [0x00905A24]
	(No symbol) [0x00932194]
	(No symbol) [0x0094B51E]
	(No symbol) [0x00931C96]
	(No symbol) [0x00903FAC]
	(No symbol) [0x00904F3D]
	GetHandleVerifier [0x00D55613+3113811]
	GetHandleVerifier [0x00D6A2DA+3199002]
	GetHandleVerifier [0x00D62AB2+3168242]
	GetHandleVerifier [0x00B03310+680016]
	(No symbol) [0x009F57ED]
	(No symbol) [0x009F2A98]
	(No symbol) [0x009F2C35]
	(No symbol) [0x009E5890]
	BaseThreadInitThunk [0x7740FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77E97C5E+286]
	RtlGetAppContainerNamedObjectPath [0x77E97C2E+238]
